# 최종 프로젝트
- 6일차

## VyOS 라우터 설치와 기본 네트워크 설정

- VyOS 라우터: 소프트웨어 기반 네트워크 라우팅, 방화벽 및 VPN 기능을 제공하는 Lunux 기반 네트워크 운영체제
- ID/PW: vyos
- **권장사양:** **512MB 램(최소), 10GB 스토리지, CPU 1개**
- 버전: 1.18버전 제공

### 설치

1. 기존과 동일하게 가상머신 생성 후 이미지 사용
2. 배포 옵션
    1. 네트워크 매핑
        1. public: VM Network
        2. internal: route-att-net
    2. 씬 프로비저닝
    3. 자동 전원 켜기 해제
3. 네트워크 어댑터 추가(총 **3개**가 되어야 함. 가능하면 순서도 맞추기)
    1. VM Network
    2. route-fw-net
    3. route-att-net
4. 세 가지 인터페이스 설정
    1. 인터넷
    2. 공격자
    3. 기업 인프라망
    4. 총 4가지 IP 대역 설정
5. NAT 변환 과정을 거쳐 외부망으로 전송시켜야 함.(NAT 설정)
    1. 외부 IP와 겹치는 경우 돌아오지 못하는 경우가 존재 → 사설 IP를 만들어 사용.
6. CD/DVD는 불필요하므로 삭제
7. 부팅

### 설정

- 부팅 확인 후 로그인
- DHCP 할당 받기
    - 동일한 IP 할당을 받고 싶을 때 설정
    - VMware 속성 설정하기
        - network editor → nat의 주소를 변경한 것. 아래의 Subnet IP를 변경 후 적용
- ip addr로 eth0~2가 맞는지 확인하기
    - MAC 주소를 보고 확인하기.
    - 설정 편집에서 네트워크 어댑터 옵션에서 MAC 주소 확인 가
- **네트워크 인터페이스 설정**
    - 유저모드: $
    - 관리자 모드: #
    - 관리자 모드 진입(설정 모드 진): configure 또는 config

```c
$ config  // 설정 모드
# set interfaces ethernet eth0 address dhcp  // DHCP 할당, 외부망
# set interfaces ethernet eth0 discription 'OUTSIDE'  // 확인하기 위한 설명
# set interfaces ethernet eth1 address '10.0.0.1/24'  // 내부망
# set interfaces ethernet eth1 discription 'INSIDE'  
# set interfaces ethernet eth2 address '10.44.44.1/24'  // 공격자
# set interfaces ethernet eth2 discription 'ATTACKER'
# commit  // 설정 적용
# save  // 저장
```

- **도메인 서버 설정**

```c
# set system name-server 8.8.8.8  // 도메인 서버: 8.8.8.8
# commit
# save
# ping yahoo.co.kr // 외부 사이트 ping 테스트. 
```

- **NAT 네트워크 설정**

```c
# set nat source rule 100 outbound-interface 'eth0'  // eth0으로 나갈 때 nat 룰 100번 설정
# set nat source rule 100 source address '10.0.0.0/8'  // 내부 IP 범위 지정
# set nat source rule 100 translation address masquerade  // IP 변경 설정
# commit
# save 
```

- **디폴트 게이트웨이 설정**
    - 설정되어 있지 않으면 트래픽이 어디로 나가야 하는지 알 수 없음 → 전달 X

```c
# set system gateway-address 192.168.111.2  //vmware에서는 게이트웨이가 2번.
# commit
# save
```

- **칼리 리눅스 네트워크 인터페이스 변경**
    - route-att-net으로 변경
    - 외부 인터넷 연결 확인
    - 저장 방법: 작성 후 esc → :wq
    sudo vim /etc/network/interfaces  // 인터페이스 아래와 같이 변경
'''
auto lo
iface lo inet loopback

auto eth0
iface eth0 inet static
address 10.44.44.44
netmask 255.255.255.0
gateway 10.44.44.1  // 기본적으로 게이트웨이는 1번
dbs-nameservers 8.8.8.8
'''	
// 저장 후	
sudo reboot
ping yahoo.co,kr

# ESXI 유저 관리
## ESXI 유저 관리

- 여러 명이 같이 프로젝트를 진행할 때 필요할 수 있다.
- ESXI를 root 계정이 아닌 새로운 유저들을 만들고 권한을 주는 방법에 대한 설명이다.
- 관리 → 보안 및 사용자 → 사용자에서 사용자 추가 가능
- 권한 만큼만 작업을 할 수 있음
- 권한 선택을 가상머신마다 줄 수 있는데, 사용자 ID로 로그인하면 그 계정에 부여된 권한만큼의 작업만 할 수 있게 된다.

- 팀원 등 각각의 권한을 적절하게 부여하고 원하는 작업을 수행할 수 있도록 해주는 게 ESXI의 유저 관리이다.